# ERC-4626: analyse performance of individual vaults

- This is an example notebook how to diagnose and debug the ignestion of data from a single vault
- Often vault have pecularities in their data, which 
    - Cause need for us to clean data (see `eth_defi.research.wrangle_vault_prices`)
    - The cleaning itself may cause further issues for other vaults
        

This is a helper notebook allowing us to diagnose these issues.




## Example vault ids

Known good and bad vaults:

```
# Fluid
VAULT_ID=1-"0x00c8a649c9837523ebb406ceb17a6378ab5c74cf" 

# Plutus
VAULT_ID="42161-0x58bfc95a864e18e8f3041d2fcd3418f48393fe6a" 

# Summer
VAULT_ID="42161-0x4f63cfea7458221cb3a0eee2f31f7424ad34bb58" 

# Ipor Base
VAULT_ID="8453-0x45aa96f0b3188d47a1dafdbefce1db6b37f58216" 

# Hype++
# https://dashboard.tenderly.co/contract/arbitrum/0x75288264fdfea8ce68e6d852696ab1ce2f3e5004/read?view=market
VAULT_ID="42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004" 
```



## Setup 

In [ ]:
import logging
import os
import pickle
from pathlib import Path
from pprint import pprint

from eth_defi.utils import setup_console_logging
from eth_defi.vault.base import VaultSpec
from eth_defi.vault.vaultdb import read_default_vault_prices, DEFAULT_RAW_PRICE_DATABASE, DEFAULT_UNCLEANED_PRICE_DATABASE, VaultDatabase

vault_id = "5000-0xE12EED61E7cC36E4CF3304B8220b433f1fD6e254".lower()

logger = logging.getLogger(__name__)

frequency = "1h"
output_folder = Path("~/.tradingstrategy/vaults").expanduser()
reader_state_db = output_folder / f"vault-reader-state-{frequency}.pickle"

spec = VaultSpec.parse_string(vault_id)

print(f"Examining vault {spec}")



Examining vault VaultSpec(chain_id=42161, vault_address='0x75288264fdfea8ce68e6d852696ab1ce2f3e5004')


## Reader state

- Check the stateful price reader state

In [2]:
from pprint import pprint

reader_states: dict[VaultSpec, dict] = pickle.load(reader_state_db.open("rb"))

if not spec in reader_states:
    raise ValueError(f"Vault {spec} not found in reader states")

state = reader_states[spec]

print(f"Vault {spec} state:")
for key, value in state.items():
    print(f"  {key}: {value}")



Vault VaultSpec(chain_id=42161, vault_address='0x75288264fdfea8ce68e6d852696ab1ce2f3e5004') state:
  last_tvl: 3704154.641178
  last_share_price: 1.408808
  max_tvl: 6241538.816345
  first_seen_at_block: 276378009
  first_block: 276382721
  first_read_at: 2024-11-20 08:20:28
  last_call_at: 2025-12-04 00:03:59
  last_block: 406904321
  peaked_at: None
  peaked_tvl: None
  faded_at: None
  entry_count: 8981
  chain_id: 42161
  vault_address: 0x75288264FDFEA8ce68e6D852696aB1cE2f3E5004
  denomination_token_address: 0xaf88d065e77c8cC2239327C5EDb3A432268e5831
  share_token_address: 0x75288264FDFEA8ce68e6D852696aB1cE2f3E5004
  one_raw_share: None
  reading_restarted_count: 0
  vault_poll_frequency: large_tvl
  token_symbol: USDC
  unsupported_token: None


## Raw price data

- Check uncleaned raw price data from scan-prices.py script

In [6]:
import pandas as pd

from eth_defi.research.wrangle_vault_prices import assign_unique_names

# Check raw price data
vault_db = VaultDatabase.read()
print(f"Checking uncleaned price data {DEFAULT_UNCLEANED_PRICE_DATABASE}")
prices_df = pd.read_parquet(DEFAULT_UNCLEANED_PRICE_DATABASE)
prices_df = assign_unique_names(vault_db.rows, prices_df, logger=lambda x: None)
vault_prices_df = prices_df.loc[prices_df["id"] == vault_id]
vault_prices_df = vault_prices_df.set_index("timestamp")

data = {
    "First timestamp": vault_prices_df.index.min(),
    "Last timestamp": vault_prices_df.index.max(),
    "Last block": f"{vault_prices_df['block_number'].iloc[-1]:,}",
    "First raw share price": vault_prices_df["share_price"].iloc[0],
    "Last raw share price": vault_prices_df["share_price"].iloc[-1],
    "Min raw share price": vault_prices_df["share_price"].min(),
    "Max raw share price": vault_prices_df["share_price"].max(),
    "Min TVL": f"${vault_prices_df['total_assets'].min():,.0f}",
    "Max TVL": f"${vault_prices_df['total_assets'].max():,.0f}",
    "Last TVL": vault_prices_df["total_assets"].iloc[-1],
    "Rows (vault)": len(vault_prices_df),
    "Rows (all)": f"{len(prices_df):,}",
}
pprint(data)


display(vault_prices_df.tail(50))

Checking uncleaned price data /Users/moo/.tradingstrategy/vaults/vault-prices-1h.parquet
{'First raw share price': 1.0,
 'First timestamp': Timestamp('2024-11-20 08:20:28'),
 'Last TVL': 3704154.641178,
 'Last block': '406,875,521',
 'Last raw share price': 1.408808,
 'Last timestamp': Timestamp('2025-12-03 22:03:55'),
 'Max TVL': '$6,211,855',
 'Max raw share price': 1.408808,
 'Min TVL': '$84,127',
 'Min raw share price': 1.0,
 'Rows (all)': '20,434,031',
 'Rows (vault)': 105}


,chain,address,block_number,share_price,total_assets,total_supply,performance_fee,management_fee,errors,id,name
timestamp,,,,,,,,,,,
2025-03-02 17:06:01,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,311533121,1.130918,6.490536e+05,5.739172e+05,NaN,NaN,,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,HYPE++
2025-03-02 19:06:03,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,311561921,1.130918,8.490527e+05,7.507638e+05,NaN,NaN,,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,HYPE++
2025-03-03 02:08:40,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,311662721,1.130918,9.022045e+05,7.977625e+05,NaN,NaN,,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,HYPE++
2025-04-04 10:27:42,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,322808321,1.160359,9.222647e+05,7.948091e+05,NaN,NaN,,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,HYPE++
2025-04-05 04:33:22,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,323067521,1.160359,9.211147e+05,7.938181e+05,NaN,NaN,,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,HYPE++
2025-04-05 08:34:24,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,323125121,1.160359,9.999506e+05,8.617590e+05,NaN,NaN,,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,HYPE++
2025-05-04 01:40:16,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,333003521,1.200659,9.557963e+05,7.960595e+05,NaN,NaN,,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,HYPE++
2025-05-05 19:56:55,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,333608321,1.200659,1.595248e+06,1.328643e+06,NaN,NaN,,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,HYPE++
2025-05-05 20:57:23,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,333622721,1.200659,2.000000e+06,1.665751e+06,NaN,NaN,,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,HYPE++


## Cleaned price data

- Check price data after cleaning



In [4]:


# Check price data
print(f"Checking cleaned price data {DEFAULT_RAW_PRICE_DATABASE}")
prices_df = read_default_vault_prices()
cleaned_df = vault_prices_df = prices_df.loc[prices_df["id"] == vault_id]

data = {
    "First timestamp": vault_prices_df.index.min(),
    "Last timestamp": vault_prices_df.index.max(),
    "Last block": f"{vault_prices_df['block_number'].iloc[-1]:,}",
    "First price": vault_prices_df["share_price"].iloc[0],
    "Last price": vault_prices_df["share_price"].iloc[-1],
    "Last price (raw)": vault_prices_df["raw_share_price"].iloc[-1],
    "Last TVL": vault_prices_df["total_assets"].iloc[-1],
    "Rows (vault)": len(vault_prices_df),
    "Rows (all)": f"{len(prices_df):,}",
    "Last timestamp (all)": prices_df.index.max(),
}
pprint(data)


print("Last 50 rows of cleaned price data:")
display(cleaned_df.tail(50))


Checking cleaned price data /Users/moo/.tradingstrategy/vaults/cleaned-vault-prices-1h.parquet
{'First price': 1.0,
 'First timestamp': Timestamp('2024-11-20 08:20:28'),
 'Last TVL': 3704154.641178,
 'Last block': '406,875,521',
 'Last price': 1.408808,
 'Last price (raw)': 1.408808,
 'Last timestamp': Timestamp('2025-12-03 22:03:55'),
 'Last timestamp (all)': Timestamp('2025-12-04 00:41:47'),
 'Rows (all)': '7,212,883',
 'Rows (vault)': 105}
Last 50 rows of cleaned price data:


,id,chain,address,block_number,share_price,total_assets,total_supply,performance_fee,management_fee,errors,name,event_count,protocol,raw_share_price,returns_1h,avg_assets_by_vault,dynamic_tvl_threshold,tvl_filtering_mask
timestamp,,,,,,,,,,,,,,,,,,
2025-03-02 17:06:01,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,311533121,1.130918,6.490536e+05,5.739172e+05,NaN,NaN,,HYPE++,723,D2 Finance,1.130918,0.000000,1.754364e+06,35087.277171,False
2025-03-02 19:06:03,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,311561921,1.130918,8.490527e+05,7.507638e+05,NaN,NaN,,HYPE++,723,D2 Finance,1.130918,0.000000,1.754364e+06,35087.277171,False
2025-03-03 02:08:40,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,311662721,1.130918,9.022045e+05,7.977625e+05,NaN,NaN,,HYPE++,723,D2 Finance,1.130918,0.000000,1.754364e+06,35087.277171,False
2025-04-04 10:27:42,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,322808321,1.160359,9.222647e+05,7.948091e+05,NaN,NaN,,HYPE++,723,D2 Finance,1.160359,0.026033,1.754364e+06,35087.277171,False
2025-04-05 04:33:22,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,323067521,1.160359,9.211147e+05,7.938181e+05,NaN,NaN,,HYPE++,723,D2 Finance,1.160359,0.000000,1.754364e+06,35087.277171,False
2025-04-05 08:34:24,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,323125121,1.160359,9.999506e+05,8.617590e+05,NaN,NaN,,HYPE++,723,D2 Finance,1.160359,0.000000,1.754364e+06,35087.277171,False
2025-05-04 01:40:16,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,333003521,1.200659,9.557963e+05,7.960595e+05,NaN,NaN,,HYPE++,723,D2 Finance,1.200659,0.034731,1.754364e+06,35087.277171,False
2025-05-05 19:56:55,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,333608321,1.200659,1.595248e+06,1.328643e+06,NaN,NaN,,HYPE++,723,D2 Finance,1.200659,0.000000,1.754364e+06,35087.277171,False
2025-05-05 20:57:23,42161-0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,42161,0x75288264fdfea8ce68e6d852696ab1ce2f3e5004,333622721,1.200659,2.000000e+06,1.665751e+06,NaN,NaN,,HYPE++,723,D2 Finance,1.200659,0.000000,1.754364e+06,35087.277171,False


In [5]:


import logging
import os
import pickle
from pathlib import Path
from pprint import pprint

import pandas as pd

from IPython.display import display

from eth_defi.research.wrangle_vault_prices import assign_unique_names

try:
    import hypersync
except ImportError as e:
    raise ImportError("Install the library with optional HyperSync dependency to use this module") from e


logger = logging.getLogger(__name__)


def main():
    setup_console_logging(
        default_log_level=os.environ.get("LOG_LEVEL", "info"),
    )

    frequency = "1h"
    output_folder = Path("~/.tradingstrategy/vaults").expanduser()
    reader_state_db = output_folder / f"vault-reader-state-{frequency}.pickle"

    vault_id = os.environ.get("VAULT_ID")
    assert vault_id is not None, "Set VAULT_ID environment variable"
    spec = VaultSpec.parse_string(vault_id)



    print("All ok")


if __name__ == "__main__":
    main()


ImportError: Install the library with optional HyperSync dependency to use this module